In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 94.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
data = pd.read_csv("Book4.csv",encoding='ISO-8859-1')
import numpy as np

In [ ]:
data.head

<bound method NDFrame.head of                       Text  Sentiment
0                 Sooo SAD          0
1              bullying me          0
2           leave me alone          0
3            Sons of ****,          0
4                      fun          1
...                    ...        ...
3999           Sleep-fail.          0
4000  What a miserable day          0
4001                gutted          0
4002                Thanks          1
4003                sucks.          0

[4004 rows x 2 columns]>

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

encoded_data = tokenizer.batch_encode_plus(
    data["Text"].tolist(),
    add_special_tokens=True,  # Add [CLS] and [SEP] tokens
    padding="max_length",
    max_length=128,
    truncation=True,
    return_attention_mask=True,
    return_tensors="pt"  # Return PyTorch tensors
)


In [ ]:
from sklearn.model_selection import train_test_split

X = encoded_data["input_ids"]
y = data["Sentiment"]

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [ ]:
print(type(X_train))
print(type(y_train))
print(type(X_val))
print(type(y_val))
print(type(X_test))
print(type(y_test))

<class 'torch.Tensor'>
<class 'pandas.core.series.Series'>
<class 'torch.Tensor'>
<class 'pandas.core.series.Series'>
<class 'torch.Tensor'>
<class 'pandas.core.series.Series'>


In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset

y_train = torch.tensor(y_train.values, dtype=torch.long)
y_val = torch.tensor(y_val.values, dtype=torch.long)
y_test = torch.tensor(y_test.values, dtype=torch.long)

train_dataset = TensorDataset(X_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = TensorDataset(X_val, y_val)
val_dataloader = DataLoader(val_dataset, batch_size=32)

test_dataset = TensorDataset(X_test, y_test)
test_dataloader = DataLoader(test_dataset, batch_size=32)


In [ ]:
from transformers import BertForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

optimizer = AdamW(model.parameters(), lr=1e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader))
epochs=3

for epoch in range(epochs):
    model.train()
    for batch in train_dataloader:
        inputs, labels = batch
        outputs = model(inputs, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

model.eval()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
# Save the fine-tuned model
model.save_pretrained("model.h5")

In [ ]:
from transformers import BertForSequenceClassification

# Load the saved model
model = BertForSequenceClassification.from_pretrained("model.h5")


In [ ]:
model.eval()

new_data = ["love ", "negative"]
inputs = tokenizer(new_data, truncation=True, padding=True, return_tensors="pt")


with torch.no_grad():
    outputs = model(**inputs)

# Get predictions (0 for negative, 1 for positive)
predictions = torch.argmax(outputs.logits, dim=1).tolist()
print(predictions)

for text, prediction in zip(new_data, predictions):
    print(f"Text: {text}")
    print(f"Sentiment Prediction: {'Positive' if prediction == 1 else 'Negative'}")


[0, 0]
Text: love 
Sentiment Prediction: Negative
Text: negative
Sentiment Prediction: Negative
